In [3]:
from typing import Union, List, Tuple

class RingElement:
    """Базовый класс для элементов колец: целых чисел и полиномов."""

    def __init__(self, data: Union[int, List[float]]):
        """
        data:
        - если это целое число → элемент кольца Z;
        - если это список коэффициентов [a0, a1, ..., an] → полином a0 + a1*x + ... + an*x^n
        """
        self.data = data
        self.is_polynomial = isinstance(data, list)

    def __repr__(self) -> str:
        if self.is_polynomial:
            terms = []
            for i, c in enumerate(self.data):
                if c != 0:
                    if i == 0:
                        terms.append(str(c))
                    elif i == 1:
                        terms.append(f"{c}*x")
                    else:
                        terms.append(f"{c}*x^{i}")
            return " + ".join(terms) if terms else "0"
        return str(self.data)

    def __eq__(self, other) -> bool:
        if not isinstance(other, RingElement):
            return False
        return self.data == other.data

def gcd_int(a: int, b: int) -> int:
    """НОД двух целых чисел по алгоритму Евклида"""
    a, b = abs(a), abs(b)
    while b != 0:
        a, b = b, a % b
    return a

def gcd_int_list(numbers: List[int]) -> int:
    """НОД списка целых чисел"""
    if not numbers:
        return 0

    result = abs(numbers[0])
    for num in numbers[1:]:
        result = gcd_int(result, abs(num))
        if result == 1:
            break
    return result

def poly_divide(f: List[float], g: List[float]) -> Tuple[List[float], List[float]]:
    """Деление полиномов с остатком"""
    f = f.copy()
    g = g.copy()

    while len(f) > 0 and f[-1] == 0:
        f.pop()
    while len(g) > 0 and g[-1] == 0:
        g.pop()

    if len(g) == 0:
        raise ValueError("Деление на нулевой полином")

    deg_f = len(f) - 1
    deg_g = len(g) - 1

    if deg_f < deg_g:
        return [0], f

    quotient = [0] * (deg_f - deg_g + 1)
    remainder = f.copy()

    for i in range(deg_f - deg_g, -1, -1):
        if len(remainder) <= i + deg_g:
            continue

        factor = remainder[i + deg_g] / g[deg_g]
        quotient[i] = factor

        for j in range(deg_g + 1):
            remainder[i + j] -= factor * g[j]

    while len(remainder) > 0 and remainder[-1] == 0:
        remainder.pop()

    return quotient, remainder

def gcd_poly(f: List[float], g: List[float]) -> List[float]:
    """НОД двух полиномов по алгоритму Евклида"""
    f, g = f.copy(), g.copy()

    while len(f) > 0 and f[-1] == 0:
        f.pop()
    while len(g) > 0 and g[-1] == 0:
        g.pop()

    if len(g) == 0:
        return f

    while len(g) > 0:
        _, remainder = poly_divide(f, g)
        if not remainder:
            break
        f, g = g, remainder

    if g and g[-1] != 0:
        normalizer = g[-1]
        g = [coeff / normalizer for coeff in g]

    return g

def gcd_poly_list(polynomials: List[List[float]]) -> List[float]:
    """НОД списка полиномов"""
    if not polynomials:
        return [0]

    result = polynomials[0].copy()
    for poly in polynomials[1:]:
        result = gcd_poly(result, poly)
        if len(result) == 1 and result[0] == 1:
            break
    return result

def gcd_ring_elements(elements: List[RingElement]) -> RingElement:
    """
    Возвращает порождающий главного идеала, порождённого заданными элементами.
    - Для Z: НОД целых чисел.
    - Для K[x]: НОД полиномов.
    """
    if not elements:
        return RingElement(0)

    is_poly = elements[0].is_polynomial
    for elem in elements:
        if elem.is_polynomial != is_poly:
            raise ValueError("Все элементы должны быть одного типа")

    if is_poly:
        polynomials = [elem.data for elem in elements]
        gcd_result = gcd_poly_list(polynomials)
        return RingElement(gcd_result)
    else:
        integers = [elem.data for elem in elements]
        gcd_result = gcd_int_list(integers)
        return RingElement(gcd_result)

def is_in_ideal(element: RingElement, generators: List[RingElement]) -> bool:
    """
    Проверяет, принадлежит ли элемент идеалу, порождённому данными генераторами.
    """
    if not generators:
        return element.data == 0

    d = gcd_ring_elements(generators)

    if d.is_polynomial:
        if not element.is_polynomial:
            return False

        _, remainder = poly_divide(element.data, d.data)
        return not remainder
    else:
        if element.is_polynomial:
            return False

        return element.data % d.data == 0


if __name__ == "__main__":

    numbers = [RingElement(12), RingElement(18), RingElement(24)]
    print(f"Идеал I = ({', '.join(str(n) for n in numbers)})")

    gcd_result = gcd_ring_elements(numbers)
    print(f"Порождающий главного идеала: {gcd_result}")
    print(f"Значит I = ({gcd_result})")

    test_cases = [6, 7, 12, 30]
    for num in test_cases:
        test_element = RingElement(num)
        belongs = is_in_ideal(test_element, numbers)
        print(f"  {num} ∈ I? {belongs}")

    polynomials = [
        RingElement([1, 2, 1]),
        RingElement([1, 1])
    ]
    print(f"Идеал I = ({', '.join(str(p) for p in polynomials)})")

    gcd_result = gcd_ring_elements(polynomials)
    print(f"Порождающий главного идеала: {gcd_result}")
    print(f"Значит I = ({gcd_result})")

    test_polys = [
        [2, 2],
        [1, 0, 1],
        [0, 3, 3],
        [1, 2, 1]
    ]

    for poly in test_polys:
        test_element = RingElement(poly)
        belongs = is_in_ideal(test_element, polynomials)
        print(f"  {test_element} ∈ I? {belongs}")

    coprime_numbers = [RingElement(3), RingElement(5)]
    print(f"Идеал I = ({', '.join(str(n) for n in coprime_numbers)})")

    gcd_result = gcd_ring_elements(coprime_numbers)
    print(f"Порождающий главного идеала: {gcd_result}")
    print("НОД(3, 5) = 1, значит идеал I = всему кольцу Z")

    test_nums = [1, 2, 7, -4]
    for num in test_nums:
        test_element = RingElement(num)
        belongs = is_in_ideal(test_element, coprime_numbers)
        print(f"  {num} ∈ I? {belongs}")


Идеал I = (12, 18, 24)
Порождающий главного идеала: 6
Значит I = (6)
  6 ∈ I? True
  7 ∈ I? False
  12 ∈ I? True
  30 ∈ I? True
Идеал I = (1 + 2*x + 1*x^2, 1 + 1*x)
Порождающий главного идеала: 1.0 + 1.0*x
Значит I = (1.0 + 1.0*x)
  2 + 2*x ∈ I? True
  1 + 1*x^2 ∈ I? False
  3*x + 3*x^2 ∈ I? True
  1 + 2*x + 1*x^2 ∈ I? True
Идеал I = (3, 5)
Порождающий главного идеала: 1
НОД(3, 5) = 1, значит идеал I = всему кольцу Z
  1 ∈ I? True
  2 ∈ I? True
  7 ∈ I? True
  -4 ∈ I? True


Идеал $(S)$, порождён множеством $S = \{s_1, s_2, \ldots, s_n\}$, $|S| = n$.

Существует элемент $y \in R$, для которого выполнено $\forall i \le n,\, y \mid s_i$ (то есть $y$ делит каждый элемент $s_i$).

Тогда можно записать
$$
s_i = y c_i, \quad c_i \in R.
$$

Любой элемент идеала $(S)$ можно записать как
$$
\sum r_i s_i = \sum r_i c_i y = y \sum r_i c_i,
$$
где $\sum r_i c_i \in R$.

Отсюда следует включение
$$
(S) \subseteq (y).
$$

Чтобы показать, что $(y) \subseteq (S)$, возьмём произвольный элемент из $(y)$:
$$
r y, \quad r \in R.
$$

Поскольку для каждого $s_i$ существует $c_i \in R$, такое что $s_i = y c_i$, то элемент $r y$ можно представить как линейную комбинацию элементов $s_i$:
$$
r y = \sum_i r_i s_i,
$$
при подходящем выборе коэффициентов $r_i \in R$ (например, если $c_i$ обратим, можно взять $r_i = r c_i^{-1}$).

Отсюда следует
$$
(y) \subseteq (S).
$$

Объединяя включения, получаем равенство
$$
(S) = (y),
$$
значит $(S)$ является главным и порождается элементом $y$.


## Применение алгоритма Евклида

Из доказанного выше следует, что идеал $(S)$ порождается элементом $y$, который делит все элементы множества $S$. Но какой элемент $y$ нужно выбрать?

Ключевое наблюдение: среди всех элементов, которые делят все $s_i$, нам нужен **наибольший общий делитель** (НОД). Это связано с тем, что:

- Любой общий делитель элементов $s_i$ порождает идеал, содержащий все $s_i$.
- Если выбрать общий делитель, который не является наибольшим, то порожденный им идеал будет содержать исходный, но будет строго больше.
- НОД — это максимальный элемент по делимости среди всех общих делителей.
- Именно идеал, порождённый НОД, совпадает с исходным идеалом $(S)$.
- Алгоритм Евклида гарантирует, что НОД представляется как линейная комбинация исходных элементов, что подтверждает его принадлежность идеалу $(S)$.

Поэтому $y = \gcd(s_1, s_2, \ldots, s_n)$.


### Случай 1: Кольцо целых чисел $\mathbb{Z}$

Пусть даны целые числа $a_1, a_2, \ldots, a_k \in \mathbb{Z}$, порождающие идеал $I = (a_1, \ldots, a_k)$.

**Утверждение:** Идеал $I$ является главным, порождаемым НОД элементов $a_i$.

**Доказательство:**

Пусть $d = \gcd(a_1, a_2, \ldots, a_k)$. По определению НОД:

- $d$ делит каждый элемент $a_i$, то есть $\forall i,\, d \mid a_i$.

- По расширенному алгоритму Евклида существуют коэффициенты $x_1, x_2, \ldots, x_k \in \mathbb{Z}$, такие что
  $$
  d = x_1 a_1 + x_2 a_2 + \cdots + x_k a_k.
  $$
  Это означает, что $d$ лежит в идеале $(a_1, \ldots, a_k)$.

- Поскольку $d$ делит все $a_i$, имеем $(a_1, \ldots, a_k) \subseteq (d)$.

- Поскольку $d \in (a_1, \ldots, a_k)$, имеем $(d) \subseteq (a_1, \ldots, a_k)$.

Следовательно,
$$
I = (a_1, \ldots, a_k) = (d) = (\gcd(a_1, \ldots, a_k)).
$$

**Алгоритм поиска:** Применяем алгоритм Евклида последовательно:
$$
d_1 = \gcd(a_1, a_2), \quad d_2 = \gcd(d_1, a_3), \quad \ldots, \quad d_{k-1} = \gcd(d_{k-2}, a_k).
$$
Результат $d_{k-1} = \gcd(a_1, \ldots, a_k)$.


### Случай 2: Кольцо полиномов $K[x]$

Пусть даны полиномы $f_1(x), f_2(x), \ldots, f_k(x) \in K[x]$, где $K$ — поле, образующие идеал $I = (f_1, \ldots, f_k)$.

**Утверждение:** Идеал $I$ является главным, порождаемым НОД полиномов $f_i(x)$.

**Доказательство:**

Пусть $d(x) = \gcd(f_1(x), f_2(x), \ldots, f_k(x))$. По определению НОД полиномов:

- $d(x)$ делит каждый полином $f_i(x)$, то есть $\forall i,\, d(x) \mid f_i(x)$ в кольце полиномов.

- По расширенному алгоритму Евклида для полиномов существуют полиномы $x_1(x), x_2(x), \ldots, x_k(x) \in K[x]$, такие что
  $$
  d(x) = x_1(x) f_1(x) + x_2(x) f_2(x) + \cdots + x_k(x) f_k(x).
  $$
  Это означает, что $d(x)$ лежит в идеале $(f_1, \ldots, f_k)$.

- Поскольку $d(x)$ делит все $f_i(x)$, имеем $(f_1, \ldots, f_k) \subseteq (d(x))$.

- Поскольку $d(x) \in (f_1, \ldots, f_k)$, имеем $(d(x)) \subseteq (f_1, \ldots, f_k)$.

Следовательно,
$$
I = (f_1, \ldots, f_k) = (d(x)) = (\gcd(f_1, \ldots, f_k)).
$$

**Алгоритм поиска:** Применяем алгоритм Евклида для полиномов:
- Делим $f_1(x)$ на $f_2(x)$, получаем остаток $r_1(x)$.
- Делим $f_2(x)$ на $r_1(x)$, получаем остаток $r_2(x)$.
- Продолжаем до нулевого остатка; последний ненулевой остаток — это $\gcd$.